In [ ]:
import numpy as np
from scipy.optimize import root
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Modèle de Brusselator

La dynamique de la réaction oscillante découverte par Belousov et Zhabotinsky, peut être modélisée par le modèle dit de Brusselator en fonction de deux paramètres :

$$
\left\{\begin{aligned}
\frac{dy_1}{dt} & = 1 - (b+1) y_1 + a y_1^2y_2\\
\frac{dy_2}{dt} & = b y_1 - a y_1^2y_2
\end{aligned}\right.
$$

In [ ]:
class brusselator_model:

    def __init__(self, a, b) :
        self.a = a
        self.b = b

    def fcn(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b
        y1_dot = 1 - (b+1)*y1 + a*y1*y1*y2
        y2_dot = b*y1 - a*y1*y1*y2
        return np.array([y1_dot, y2_dot])

    def jac(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b
        return np.array([[2*a*y1*y2 -(b+1), a*y1*y1], [-2*a*y1*y2 + b, -a*y1*y1]])

##  Solution quasi-exacte

La solution quasi-exacte est obtenue en utilisant une méthode Runge-Kutta explicite d'ordre 5 avec contrôle du pas et tolérances fines appelée méthide de Dormand et Prince.

In [ ]:
bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

tini = 0. 
tend = 20.

yini = (1.5, 3)

sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[0], mode="lines", name="y1"))
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[1], mode="lines", name="y2"))
fig.update_layout(title="Solution quasi-exacte du système du Brusselator", xaxis_title="t")
fig.show()

## Caractérisation de la raideur

In [ ]:
jac = bm.jac

eig_vals = np.zeros((sol_exa.t.size, 2), dtype=np.complex_)
for it in range(sol_exa.t.size):
    eig_vals[it], _ = np.linalg.eig(jac(0, sol_exa.y[:,it]))

lambda1 = eig_vals[:, 0]
lambda2 = eig_vals[:, 1]

fig_eig = make_subplots(rows=2, cols=1, subplot_titles=("Valeurs propres réelles", "Valeurs propres imaginaires"))
fig_eig.add_trace(go.Scatter(x=sol_exa.t, y=np.real(lambda1), mode="lines", name="lamba1", line_color="blue"), row=1, col=1)
fig_eig.add_trace(go.Scatter(x=sol_exa.t, y=np.real(lambda2), mode="lines", name="lamba2", line_color="red"), row=1, col=1)
fig_eig.add_trace(go.Scatter(x=sol_exa.t, y=np.imag(lambda1), mode="lines", name="y1", line_color="blue",
                             showlegend=False), row=2, col=1)
fig_eig.add_trace(go.Scatter(x=sol_exa.t, y=np.imag(lambda2), mode="lines", name="y2", line_color="red",
                             showlegend=False), row=2, col=1)
fig_eig.update_layout(xaxis_title="t", height=600)
fig_eig.show()

## Méthode d'Euler explicite

In [ ]:
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

In [ ]:
fig = make_subplots(rows=2, cols=1)

nt = np.arange(1000, 10001, 1000)
for nt_i in nt:
    sol_fe = forward_euler(tini, tend, nt_i, yini, fcn)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_fe.t)
    y1_err_fe = np.abs(sol_exa.y[0] - sol_fe.y[0])
    y2_err_fe = np.abs(sol_exa.y[1] - sol_fe.y[1])    
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=sol_fe.y[0], mode="lines", name="y1"), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=sol_fe.y[1], mode="lines", name="y2"), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=y1_err_fe, mode="lines", name="err y1"), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=y2_err_fe, mode="lines", name="err y2"), row=2, col=1)
    
fig.data[0].visible = True
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True

# Create and add slider
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f"{nt_i}",
                args=[{"visible": [el==4*i or el==4*i+1 or el==4*i+2 or el==4*i+3 for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={"prefix": "nt : "}, steps=steps)]

fig.update_yaxes(type='log', exponentformat='e', row=2, col=1)
fig.update_layout(sliders=sliders, height=800)
fig.show()

## Méthode d'Euler implicite

In [ ]:
def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

In [ ]:
fig = make_subplots(rows=2, cols=1)

nt = np.arange(1000, 10001, 1000)
for nt_i in nt:
    sol_fe = backward_euler(tini, tend, nt_i, yini, fcn)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_fe.t)
    y1_err_fe = np.abs(sol_exa.y[0] - sol_fe.y[0])
    y2_err_fe = np.abs(sol_exa.y[1] - sol_fe.y[1])    
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=sol_fe.y[0], mode="lines", name="y1"), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=sol_fe.y[1], mode="lines", name="y2"), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=y1_err_fe, mode="lines", name="err y1"), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=y2_err_fe, mode="lines", name="err y2"), row=2, col=1)
    
fig.data[0].visible = True
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True

# Create and add slider
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f"{nt_i}",
                args=[{"visible": [el==4*i or el==4*i+1 or el==4*i+2 or el==4*i+3 for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={"prefix": "nt : "}, steps=steps)]

fig.update_yaxes(type='log', exponentformat='e', row=2, col=1)
fig.update_layout(sliders=sliders, height=800)
fig.show()